## Standardised Experiment analysis example: My Experiment

In [1]:
import pandas as pd
import numpy as np
import psycopg2
import time
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
import seaborn as sns
import statsmodels.stats.power as smp
from statsmodels.stats.power import NormalIndPower
import nbconvert
from IPython.display import HTML, display, Math, Latex, Image

HTML('''<script src='https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.5/MathJax.js?config=TeX-MML-AM_CHTML' async></script>''')
HTML('''
<script>code_show=true; function code_toggle(){if (code_show){$('div.input').hide();}else{$('div.input').show();}code_show=!code_show}$( document ).ready(code_toggle);</script>
The raw code for this IPython notebook is by default hidden for easier reading. 
Raw code toggle: click <a href="javascript:code_toggle()">here.</a>''')

## Part 1: 
* Create a detailed write-up of how to analyse experiments statistically, 
* Explain the methods used in notebook markdown 
* Give commented versions the necessary python functions to show how to apply them.

## 1.1 Your Experiment

In [2]:
#Configure this cell to read the experiment properties automatically
print('Your experiment Name is: my_experiment')
print("Your experiment variants are: ['v1' 'v0']")
print('Your experiment launched on: 2020-01-02')

Your experiment Name is: my_experiment
Your experiment variants are: ['v1' 'v0']
Your experiment launched on: 2020-01-02


### 1.1.1 Experiment User Allocation
When users are allocated to an experiment they are evenly split into groups or cohorts, depending on the number of variants in the experiment (v0, v1, v2 ... etc). Users will be distributed evenly between the variants.

### 1.1.2 Experiment User and Order Data  
Query the following data to analyse the experiment overall by country and device  
Fields:  
* COUNT(has_order) as users  
* SUM(has_order) as customers   
* Grouped by:   variant_id, country, device  

In [3]:
exp_df = pd.read_csv('exp_df_cr.csv')

In [4]:
exp_df

,experiment_id,variant_id,users,customers
0,rightmove-homeservices-saleslandingpage,0,1950,11
1,rightmove-homeservices-saleslandingpage,1,2053,20


### 1.2 Calculating the Success of an experiment variant
The success is measured by looking at how well users in each variant convert.  
This is done by calculating a proportion for each variant (or, in fact, the mean) that represents the sample of users in that variant, which we normally call Conversion Rate. For each variant, Conversion Rate is given by:

$$CR_0 = \frac{\text{Number of Users Converting in cohort v0 within N Days}} {\text{Number of Unique Users (UVs) in cohort v0}}$$

Or: 

$$CR_0 = \frac{\text{Conversions_v0}} {\text{UVs_v0}}$$

In [5]:
# Example Pyhton function for calculating Conversion Rate
def var_cr(df):
    conversions = float(sum(df.customers))
    uvs = float(sum(df.users))
    cr = float(conversions / uvs)
    return cr
# Returns the conversion rate from a given dataframe

#### 1.2.1 Calculate the Conversion Rates of this experiment

In [6]:
print('Variant 0 Conversion Rate =',round(var_cr(exp_df[exp_df.variant_id==0]),4)*100,'%')
print('Variant 1 Conversion Rate =',round(var_cr(exp_df[exp_df.variant_id==1]),4)*100,'%')

Variant 0 Conversion Rate = 0.5599999999999999 %
Variant 1 Conversion Rate = 0.97 %


### 1.3 Hypothesis and measuring the success of an experiment
Our hypothesis (H1) is that one variant will be more successful or less successful than another, which we will detect by observing a change in conversion rate for that group of users compared to a control group. This change in conversion rate is called "uplift" (also, the delta or effect size). 

To calculate the uplift in conversion rate we subtract the variant 1 conversion rate (v1) from the control conversion rate (v0), and divide by the v0 conversion to give the relative lift:

$$
Lift = \frac{CR_1-CR_0}{CR_0}
$$

In [7]:
# Example Python function for calculating lift between two variants, v0 and v1
def exp_lift(df):
    df_0 = df[df.variant_id==0]
    df_1 = df[df.variant_id==1]
    cr_0 = var_cr(df_0)
    cr_1 = var_cr(df_1)
    lift = (cr_1 - cr_0) / cr_0
    return lift*100, cr_0, cr_1

#### 1.3.1 Calculate the Lift of this experiment

In [8]:
print('Variant 1 Lift =',round(exp_lift(exp_df)[0],2),'%')

Variant 1 Lift = 72.7 %


### 1.4 Robust experimental approach
In order to determine if the uplift observed in our sample of users is true, and we have not observed a difference between the two variants that isn't really there (eg noise in the data), we want to control for the likelihood of getting a false discovery (or Type 1 error).  
* The probability of committing a Type I error is called the **significance level**, and is denoted by the symbol α.

#### 1.4.1 Setting the alpha (α) or significance level
The alpha sets the standard for how extreme the data must be before we can reject the null hypothesis. 
For example, if the alpha level is set at .05, this means we may reject the null only if the observed data are so unusual that they would have occurred by chance at most 5 % of the time.  
* We set the value for the alpha before running each experiment, and the smaller the value of alpha the lower the false discovery rate (FDR) and the more robust the experiment result will be.

#### 1.4.2 We do this by using an hypothesis test
Hypothesis testing is a way for you to figure out if results from a test are valid or repeatable.  
For example, if someone said they had found a new drug that cures cancer, you would want to be sure it was probably true. We use an hypothesis test to measure the likelihood that the observation (curing cancer in a drug trial with limited sample size) is likely to be true when applied to the larger population.  

#### 1.4.3 Why use a Z-test?
A Z-test is any statistical test for which the distribution of the test statistic under the null hypothesis can be approximated by a normal distribution (or Gaussian or Laplace–Gauss distribution).

<img src="Normal_Distribution.png" width=500 />

In Experimentation, a Z-test is used because conversion is binomial and approximately normally distributed (Conversion is 0 if a Unique visitor visits but does not convert, or 1 if they visit and convert).

The Z-test returns a Z-score which is the number of standard deviations below or above the population mean a measurement is. A z-score is also known as a standard score as it can be placed on a normal distribution curve. Z-scores are a way to compare results from a test to a “normal” population.

#### 1.4.3 Calculating a Z-score
The Z score for an experiment is given by the following formula for a two proportion hypothesis test:
$$
Z=\frac{CR_0 - CR_1}{SE}
$$

Where $CR_0$ is the conversion rate of the control group (v0), $CR_1$ is the conversion rate of the variant group (v1), and SE is the pooled standard error, given by the following formula:
$$
SE = \sqrt{se_0^2 + se_1^2}
$$
Where the standard error of variant 0 is:
$$
se_0 = \frac{\sigma_0}{\sqrt n_0}
$$
And $\sigma_0$ (sigma) is the standard deviation and $n_0$ is the number of users.
$$
\sigma_0 =  \sqrt{ \sum CR_0(1-CR_0) }
$$
So that:
$$
se_0 = \sqrt{  \frac {CR_0(1-CR_0)} {\sum n_0 } }
$$

Replacing into the formula for the z score this gives:
$$
Z=\frac{CR_0 - CR_1}{\sqrt{se_0^2 + se_1^2}}
$$

In [9]:
# Example Pyhton function for calculating Standard Error

def var_se(df):
    se = float(np.sqrt(var_cr(df)*(1-var_cr(df)) / sum(df.users)))
    return se

# Example Pyhton function for calculating Z score

def exp_z_score(df):
    df_0 = df[df.variant_id==0]
    df_1 = df[df.variant_id==1]
    cr_0 = var_cr(df_0)
    se_0 = var_se(df_0)
    cr_1 = var_cr(df_1)
    se_1 = var_se(df_1)
    z_score = (cr_0 - cr_1) / (np.sqrt((se_0)**2 + (se_1)**2))
    return z_score

### 1.5 Reporting on the success of an experiment (or, can we reject the null hypothesis?)
We convert the Z score to a p-value which gives a measurement of the strength of evidence in support of a null hypothesis:

* If the p-value is **less than or equal** to the significance level or alpha **(p < α)**, then we reject the null hypothesis, and we say the **result is statistically significant**.
* If the p-value is **greater than** alpha **(p > α)**, then we fail to reject the null hypothesis, and we say that the **result is statistically not significant**. 

P values are obtained using a table for converting the Z score to a p value (http://www.z-table.com/), which can be done easily in code in python or R using the right command/package: p_value = stats.norm.sf(abs(z_score))*2

### 1.5.1 Experiment Confidence level or p-value
Having obtained a p-value for the experiment, this is normally converted to a "confidence level" to ease communication of the result, by subtracting the p value from 1, like this:
$$
p = 0.17
$$
$$
Confidence = (1-0.17)\cdot100 = 83\%
$$

In [10]:
alpha = 0.1
confidence = (1- alpha)*100

def exp_pvalue(z_score):
    p_value = stats.norm.sf(abs(z_score))*2
    return p_value

def exp_significant(p):
    if p <= alpha: 
          print('Expriment variant is Significant \n')
    else:
          print('Expriment variant is NOT Significant\n')
            
def experiment_overview(df):  
    lift = exp_lift(df)[0]
    z_score = exp_z_score(df)
    p_value = exp_pvalue(exp_z_score(df))
    cr_0 = exp_lift(df)[1]
    cr_1 = exp_lift(df)[2]
    nobs1 = exp_df[exp_df.variant_id==0].users
    #significant = exp_significant(p_value)
    output = [round(cr_0,5), round(cr_1,5), 
          round(z_score,5), round(p_value,5), 
          round((1-p_value)*100,2), round(lift, 5), nobs1]
    df_cr = pd.DataFrame([output])
    df_cr.columns = ['CR Control', 'CR Contender','z-score', 'P-value', 'Significance', 'Lift','nobs1']
    return df_cr

#### 1.5.2 Calculating the Experiment confidence

In [11]:
exp_overview_df = experiment_overview(exp_df)
print('Experiment Variant 1 Confidence =',round(exp_overview_df['Significance'][0],0),'%')

Experiment Variant 1 Confidence = 86.0 %


## Part 2: 
* Explore methods for calculating margin of error for each conversion rate in an experiment 
* Use those intervals to create intervals for the experiment uplift, such that when an experiment reaches 90% confidence, the intervals can be used to report on the experiment result intuitively 


### Part 2: Using Margin of error to simplify experiment reporting

#### 2.1 What is Margin of Error?

A margin of error tells you how many percentage points your results will differ from the real population value. For example, a 95% confidence interval with a 4 percent margin of error means that your statistic will be within 4 percentage points of the real population value 95% of the time.  

The Margin of Error can be calculated by:  

* Margin of error = Critical Z value x Standard error of the statistic

And as we have already calculated the Standard Error for each variant, this just means we need to calculate the Critical Z Value for the confidence interval of interest.

#### 2.2 What is the critical value? 
The critical value is the Z score for a specific p-value (or confidence interval), and is given by $Z_c$.
In this case we want the Z score for a p-value of 0.1, given the the alpha or significane level we set before running the experiment. This will therefore give margin or error for the 90% confidence interval once calculated ( as Confidence = (1-p) ).

Therefore:
$$
    p_c = 0.1
$$
$$
    Z_c = stats.norm.ppf(p_c) = stats.norm.ppf(0.1)
$$
Thus:
$$
    Z_c = 1.2815515655446004
$$

Having obtained the required Z_critical, we now have everything we need to calculate the Margin of Error for the 90% confidence interval:

$$
ME_0= Z_c \cdot \frac{CR_0 - CR_1}{\sqrt{se_0^2 + se_1^2}}
$$

And as:
$$
se_0 = \sqrt{  \frac {CR_0(1-CR_0)} {\sum n_0 } }
$$

This gives:
$$
ME_0= Z_c \cdot se_0
$$



In [12]:
def exp_critical(df):
    df_0 = exp_df[exp_df.variant_id==0]
    df_1 = exp_df[exp_df.variant_id==1]
    cr_0 = var_cr(df_0)
    se_0 = var_se(df_0)
    cr_1 = var_cr(df_1)
    se_1 = var_se(df_1)
    p_critical = alpha
    z_score_critical = abs(stats.norm.ppf(abs(p_critical/2)))
    return z_score_critical, se_0, se_1

In [13]:
print('Variant 0 Conversion Rate =',round(var_cr(exp_df[exp_df.variant_id==0])*100,2),'%')
print('Variant 1 Conversion Rate =',round(var_cr(exp_df[exp_df.variant_id==1])*100,2),'%')
print('Variant 0 Standard Error =',round(var_se(exp_df[exp_df.variant_id==0])*100,2))
print('Variant 1 Standard Error =',round(var_se(exp_df[exp_df.variant_id==1])*100,2))

Variant 0 Conversion Rate = 0.56 %
Variant 1 Conversion Rate = 0.97 %
Variant 0 Standard Error = 0.17
Variant 1 Standard Error = 0.22


In [14]:
margin_of_error_0 = exp_critical(exp_df)[0] * exp_critical(exp_df)[1]

In [15]:
margin_of_error_1 = exp_critical(exp_df)[0] * exp_critical(exp_df)[2]

In [16]:
print('Variant 0 Margin of Error =',round(margin_of_error_0*100,3),'%')
print('Variant 1 Margin of Error =',round(margin_of_error_1*100,3),'%')

Variant 0 Margin of Error = 0.279 %
Variant 1 Margin of Error = 0.357 %


#### 2.3 How do we use it to calculate Lift intervals?
To estimate a difference in population proportions (or a treatment effect), the statistic is a difference in sample proportions. So the confidence interval is

$$\textrm { Confidence Interval = sample statistic } \pm \textrm{ Margin of error} $$

#### 2.3.1 Calculate the Confidence interval of each conversion rate
* The confidence interval for each conversion rate is given by 

Upper Confidence Interval for $CR_0$ = 
$$
CI_{0\_Upper} = CR_0 + ME_0
$$

Upper Confidence Interval for $CR_0$ = 
$$
CI_{0\_Lower} = CR_0 - ME_0
$$

* This is then repeated for the other variant, leading to 4 calculated intervals in total (two upper, two lower)

In [17]:
confidence_interval_0 = (var_cr(exp_df[exp_df.variant_id==0]) - margin_of_error_0,
                       var_cr(exp_df[exp_df.variant_id==0]) + margin_of_error_0)

In [18]:
confidence_interval_1 = (var_cr(exp_df[exp_df.variant_id==1]) - margin_of_error_1,
                       var_cr(exp_df[exp_df.variant_id==1]) + margin_of_error_1)

In [19]:
print('Variant 0 Lower Interval =',round(confidence_interval_0[0]*100,2),'%')
print('Variant 0 Upper Interval =',round(confidence_interval_0[1]*100,2),'%')
print('Variant 1 Lower Interval =',round(confidence_interval_1[0]*100,2),'%')
print('Variant 1 Upper Interval =',round(confidence_interval_1[1]*100,2),'%')

Variant 0 Lower Interval = 0.29 %
Variant 0 Upper Interval = 0.84 %
Variant 1 Lower Interval = 0.62 %
Variant 1 Upper Interval = 1.33 %


#### 2.3.2 Calculate the Lift Intervals:
* Calculate the upper and lower values the lift could take based on the conversion rate confidence intervals

And as:
$$
Lift = \frac{CR_1-CR_0}{CR_0}
$$

Lift Upper Bound = 
$$
Lift_{Upper} = \frac{CI_{1\_Upper}-CI_{0\_Lower}}{CI_{0\_Lower}}
$$

Lift Lower Bound = 
$$
Lift_{Lower} = \frac{CI_{1\_Lower}-CI_{0\_Upper}}{CI_{0\_Upper}}
$$

In [20]:
lift_upper = ((confidence_interval_1[1] - confidence_interval_0[0]) / confidence_interval_0[0])

In [21]:
lift_lower = ((confidence_interval_1[0] - confidence_interval_0[1]) / confidence_interval_0[1])

In [22]:
print('Lift Upper Interval =',round(lift_upper*100,2),'%')
print('Lift Lower Interval =',round(lift_lower*100,2),'%')

Lift Upper Interval = 366.71 %
Lift Lower Interval = -26.74 %


In [23]:
HTML('''Raw code toggle: click <a href="javascript:code_toggle()">here.</a>''')